# TIME ETS

In [ ]:
from statsmodels.tsa.seasonal import STL

from sktime.forecasting.ets import AutoETS
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

from utils import time_plot, get_figure, fit_and_forecast, plot_fit_and_forecast
from data import get_apple_5y, get_switzerland_temperature, get_france_death_rate_20y

## Simple Exponential Smoothing (SES)

In [ ]:
data = get_apple_5y().resample('W').mean().ffill()['Close']
train_size = int(len(data) * 0.8)
fig, ax = time_plot(
    x=data.index,
    y=data,
    label="Signal",
    title='Weekly Apple Inc. (AAPL) Closing Prices',
    xlabel='Date',
    ylabel='Closing Price (USD)',
    return_fig=True,
)
ax.axvline(x=data.index[train_size], color='gray', linestyle='--', label='Train/Test Split')

for alpha in [.1,.8]:
    ses = ExponentialSmoothing(smoothing_level=alpha, optimized=False)
    fitted_values, pred, _ = fit_and_forecast(ses, data, train_size)
    plot_fit_and_forecast(ax, fitted_values, pred, model_name=f"SES ($\\alpha={alpha}$)")
ax.legend()

In [ ]:
data = get_apple_5y().resample('W').mean().ffill()['Close']
train_size = int(len(data) * 0.8)
fig, axs = get_figure(ncols=2, sharey=True)
for ax, error in zip(axs, ['add', 'mul']):
    time_plot(
        x=data.index,
        y=data,
        label="Signal",
        title='Error Type: ' + error,
        xlabel='Date',
        ylabel='Closing Price (USD)',
        ax=ax,
    )
    ax.axvline(x=data.index[train_size], color='gray', linestyle='--')
    
    ets = AutoETS(error=error, auto=False, trend=None, seasonal=None)
    fitted_values, pred, pred_interval = fit_and_forecast(ets, data, train_size)
    alpha = ets._fitted_forecaster.smoothing_level
    plot_fit_and_forecast(ax, fitted_values, pred, pred_interval, model_name=f"ETS({error[0].upper()},N,N) ($\\alpha={alpha}$)")
    ax.legend(loc='upper left', ncols=2)
fig.tight_layout()

In [ ]:
data = get_switzerland_temperature().set_index('dt').asfreq('ME')['AverageTemperature']
train_size = int(len(data) * 0.8)
fig, ax = time_plot(
    x=data.index,
    y=data,
    label="Signal",
    title='Monthly Average Temperature in Switzerland',
    xlabel='Year',
    ylabel='Average Temperature (°C)',
    return_fig=True,
)
ax.axvline(x=data.index[train_size], color='gray', linestyle='--', label='Train/Test Split')

for alpha in [.1,.8]:
    ses = ExponentialSmoothing(smoothing_level=alpha, optimized=False)
    fitted_values, pred, _ = fit_and_forecast(ses, data, train_size)
    plot_fit_and_forecast(ax, fitted_values, pred, model_name=f"SES ($\\alpha={alpha}$)")
ax.legend(ncol=4)

In [ ]:
data = get_switzerland_temperature().set_index('dt').asfreq('ME')['AverageTemperature']
data = data+10
train_size = int(len(data) * 0.8)
fig, axs = get_figure(ncols=2, sharey=True)
for ax, error in zip(axs, ['add', 'mul']):
    time_plot(
        x=data.index,
        y=data,
        label="Signal",
        title='Error Type: ' + error,
        xlabel='Year',
        ylabel='Average Temperature (°C) + 10°C',
        ax=ax
    )
    ax.axvline(x=data.index[train_size], color='gray', linestyle='--')
    
    ets = AutoETS(error=error, auto=False, trend=None, seasonal=None)
    fitted_values, pred, pred_interval = fit_and_forecast(ets, data, train_size)
    alpha = ets._fitted_forecaster.smoothing_level
    plot_fit_and_forecast(ax, fitted_values, pred, pred_interval, model_name=f"ETS({error[0].upper()},N,N) ($\\alpha={alpha}$)")
    ax.legend(loc='upper left', ncols=2)
fig.tight_layout()

## Holt's linear trend method

In [ ]:
data = get_apple_5y().resample('W').mean().ffill()['Close']
train_size = int(len(data) * 0.8)
fig, ax = time_plot(
    x=data.index,
    y=data,
    label="Signal",
    title='Weekly Apple Inc. (AAPL) Closing Prices',
    xlabel='Date',
    ylabel='Closing Price (USD)',
    return_fig=True,
)
ax.axvline(x=data.index[train_size], color='gray', linestyle='--', label='Train/Test Split')

for alpha in [.1]:
    for beta in [.1,.8]:
        holt = ExponentialSmoothing(smoothing_level=alpha, smoothing_trend=beta, trend='add', optimized=False)
        fitted_values, pred, _ = fit_and_forecast(holt, data, train_size)
        plot_fit_and_forecast(ax, fitted_values, pred, model_name=f"Holt linear trend ($\\alpha={alpha}$, $\\beta^*={beta}$)")
ax.legend()

In [ ]:
data = get_apple_5y().resample('W').mean().ffill()['Close']
train_size = int(len(data) * 0.8)
fig, axs = get_figure(ncols=2, sharey=True)
for ax, error in zip(axs, ['add', 'mul']):
    time_plot(
        x=data.index,
        y=data,
        label="Signal",
        title='Error Type: ' + error,
        xlabel='Date',
        ylabel='Closing Price (USD)',
        ax=ax,
    )
    ax.axvline(x=data.index[train_size], color='gray', linestyle='--')
    
    ets = AutoETS(error=error, auto=False, trend='add', seasonal=None)
    fitted_values, pred, pred_interval = fit_and_forecast(ets, data, train_size)
    alpha = ets._fitted_forecaster.smoothing_level
    beta = ets._fitted_forecaster.smoothing_trend
    plot_fit_and_forecast(ax, fitted_values, pred, pred_interval, model_name=f"ETS({error[0].upper()},A,N) ($\\alpha={alpha:.2f}$, $\\beta^*={beta:.2e}$)")
    ax.legend()
fig.tight_layout()

## Damped Holt's linear trend method

In [ ]:
data = get_apple_5y().resample('W').mean().ffill()['Close']
train_size = int(len(data) * 0.8)
fig, ax = time_plot(
    x=data.index,
    y=data,
    label="Signal",
    title='Weekly Apple Inc. (AAPL) Closing Prices',
    xlabel='Date',
    ylabel='Closing Price (USD)',
    return_fig=True,
)
ax.axvline(x=data.index[train_size], color='gray', linestyle='--', label='Train/Test Split')

for alpha in [.1]:
    for beta in [.8]:
        for phi in [.4,.95]:
            holt = ExponentialSmoothing(smoothing_level=alpha, smoothing_trend=beta, damping_trend=phi,  trend='add', damped_trend=True, optimized=False)
            fitted_values, pred, _ = fit_and_forecast(holt, data, train_size)
            plot_fit_and_forecast(ax, fitted_values, pred, model_name=f"Damped Holt linear trend ($\\alpha={alpha}$, $\\beta^*={beta}$, $\\phi={phi}$)")
ax.legend()

In [ ]:
data = get_apple_5y().resample('W').mean().ffill()['Close']
train_size = int(len(data) * 0.8)
fig, axs = get_figure(ncols=2, sharey=True)
for ax, error in zip(axs, ['add', 'mul']):
    time_plot(
        x=data.index,
        y=data,
        label="Signal",
        title='Error Type: ' + error,
        xlabel='Date',
        ylabel='Closing Price (USD)',
        ax=ax,
    )
    ax.axvline(x=data.index[train_size], color='gray', linestyle='--')
    
    ets = AutoETS(error=error, auto=False, trend='add', damped_trend=True, seasonal=None)
    fitted_values, pred, pred_interval = fit_and_forecast(ets, data, train_size)
    alpha = ets._fitted_forecaster.smoothing_level
    beta = ets._fitted_forecaster.smoothing_trend
    phi = ets._fitted_forecaster.damping_trend
    plot_fit_and_forecast(ax, fitted_values, pred, pred_interval, model_name=f"ETS({error[0].upper()},Ad,N) ($\\alpha={alpha}$, $\\beta^*={beta:.1e}$, $\\phi={phi}$))")
    ax.legend()
fig.tight_layout()

## Holt-Winters' additive method

In [ ]:
data = get_switzerland_temperature().set_index('dt').asfreq('ME')['AverageTemperature']
train_size = int(len(data) * 0.8)
fig, ax = time_plot(
    x=data.index,
    y=data,
    label="Signal",
    title='Monthly Average Temperature in Switzerland',
    xlabel='Year',
    ylabel='Average Temperature (°C)',
    return_fig=True,
)
ax.axvline(x=data.index[train_size], color='gray', linestyle='--', label='Train/Test Split')

for alpha in [.1]:
    for beta in [.4]:
        for gamma in [.1,.8]:
            holt_winter = ExponentialSmoothing(smoothing_level=alpha,  trend='add', smoothing_trend=beta, seasonal='add', smoothing_seasonal=gamma, optimized=False, sp=12)
            fitted_values, pred, _ = fit_and_forecast(holt_winter, data, train_size)
            plot_fit_and_forecast(ax, fitted_values, pred, model_name=f"Holt-Winter ($\\alpha={alpha}$, $\\beta^*={beta}$, $\\gamma={gamma}$)")
ax.legend(ncol=2)

In [ ]:
data = get_switzerland_temperature().set_index('dt').asfreq('ME')['AverageTemperature']
data = data+10
train_size = int(len(data) * 0.8)
fig, axs = get_figure(ncols=2, sharey=True)
for ax, error in zip(axs, ['add', 'mul']):
    time_plot(
        x=data.index,
        y=data,
        label="Signal",
        title='Error Type: ' + error,
        xlabel='Year',
        ylabel='Average Temperature (°C) + 10°C',
        ax=ax,
    )
    ax.axvline(x=data.index[train_size], color='gray', linestyle='--')
    
    ets = AutoETS(error=error, auto=False, trend='add', seasonal='add', sp=12)
    fitted_values, pred, pred_interval = fit_and_forecast(ets, data, train_size)
    alpha = ets._fitted_forecaster.smoothing_level
    beta = ets._fitted_forecaster.smoothing_trend
    gamma = ets._fitted_forecaster.smoothing_seasonal
    plot_fit_and_forecast(ax, fitted_values, pred, pred_interval, model_name=f"ETS({error[0].upper()},A,A) ($\\alpha={alpha:.1e}$, $\\beta^*={beta:.1e}$, $\\gamma={gamma:.1e}$)")
    ax.legend(loc="center left")
fig.tight_layout()

In [ ]:
data = get_france_death_rate_20y().set_index('time').asfreq('ME')['value']
train_size = int(len(data) * 0.8)
fig, ax = time_plot(
    x=data.index,
    y=data,
    label="Signal",
    title='Monthly Death Rates Over the Last 20 Years in France',
    xlabel='Year',
    ylabel='Monthly Death Rate',
    return_fig=True,
)
ax.axvline(x=data.index[train_size], color='gray', linestyle='--', label='Train/Test Split')

for alpha in [.1]:
    for beta in [.4]:
        for gamma in [.8]:
            for season_type in ['add', 'mul']:
                holt_winter = ExponentialSmoothing(smoothing_level=alpha,  trend='add', smoothing_trend=beta, seasonal=season_type, smoothing_seasonal=gamma, optimized=False, sp=12)
                fitted_values, pred, _ = fit_and_forecast(holt_winter, data, train_size)
                plot_fit_and_forecast(ax, fitted_values, pred, model_name=f"{season_type} Holt-Winter ($\\alpha={alpha}$, $\\beta^*={beta}$, $\\gamma={gamma}$")
ax.legend(ncol=2)

In [ ]:
data = get_france_death_rate_20y().set_index('time').asfreq('ME')['value']
train_size = int(len(data) * 0.8)
fig, axs = get_figure(ncols=2, sharey=True)
for ax, error in zip(axs, ['add', 'mul']):
    time_plot(
        x=data.index,
        y=data,
        label="Signal",
        title='Error Type: ' + error,
        xlabel='Year',
        ylabel='Monthly Death Rate',
        ax=ax
    )
    ax.axvline(x=data.index[train_size], color='gray', linestyle='--')
    
    ets = AutoETS(error=error, auto=False, trend='add', seasonal='mul', sp=12)
    fitted_values, pred, pred_interval = fit_and_forecast(ets, data, train_size)
    alpha = ets._fitted_forecaster.smoothing_level
    beta = ets._fitted_forecaster.smoothing_trend
    gamma = ets._fitted_forecaster.smoothing_seasonal
    plot_fit_and_forecast(ax, fitted_values, pred, pred_interval, model_name=f"ETS({error[0].upper()},A,M) ($\\alpha={alpha:.1e}$, $\\beta^*={beta:.1e}$, $\\gamma={gamma:.1e}$)")
    ax.legend()
fig.tight_layout()

## Time series decomposition

In [ ]:
data = get_france_death_rate_20y().set_index('time').asfreq('ME')['value']
holt_winter = ExponentialSmoothing(trend='add', seasonal='add', sp=12, initialization_method='estimated')
holt_winter.fit(data)
fig, axs = get_figure(4, 1, (8, 8), grid=False)
axs[0].plot(data.index, holt_winter._fitted_forecaster.trend)
axs[0].set_ylabel('Trend')
axs[1].plot(data.index, holt_winter._fitted_forecaster.level)
axs[1].set_ylabel('Level')
axs[2].plot(data.index, holt_winter._fitted_forecaster.season)
axs[2].set_ylabel('Season')
axs[3].scatter(data.index, holt_winter._fitted_forecaster.resid)
axs[3].set_ylabel('Residual')
for ax in axs:
    ax.margins(x=0)
fig.suptitle("Holt-Winter Components: Monthly Death Rates Over the Last 20 Years in France", y=1)
fig.tight_layout()

In [ ]:
data = get_france_death_rate_20y().set_index('time').asfreq('ME')['value']
res = STL(data).fit()
fig = res.plot()
fig.axes[0].set_title("")
fig.set_size_inches(12, 8)
fig.suptitle("STL decomposition: Monthly Death Rates Over the Last 20 Years in France", y=1)
fig.set_size_inches(8,8)
fig.tight_layout()